# Model1
Model : $y = ax + b$이라 하자. \
Loss : $(y - \hat{y})^2$ where $\hat{y} = ax + b$ \
Loss L과 input data $(x,y)$ 대하여 에 대하여 ${\operatorname{d}\!L\over\operatorname{d}\!a}$ ${\operatorname{d}\!L\over\operatorname{d}\!b}$를 구하여 다음과 같이 weight : $a,b$를 업데이트 한다.\
$a = a - lr * {\operatorname{d}\!L\over\operatorname{d}\!a}$ \
$b = b - lr * {\operatorname{d}\!L\over\operatorname{d}\!b}$ \
이때 lr은 learning rate로 기본값을 0.01로 한다.

In [1]:
class Model:
    def __init__(self, a, b):
        self.a = a
        self.b = b

    def __call__(self, x):
        a = self.a
        b = self.b
        return self.y_hat(a, b, x)

    def y_hat(self, a, b, x):
        return a * x + b

    def loss(self, a, b, x, y):
        return (y - self.y_hat(a, b, x)) ** 2

    def gradient(self, x, y, h=1e-8):
        a, b = self.a, self.b
        gradient_a = (self.loss(a + h, b, x, y) - self.loss(a, b, x, y)) / h
        gradient_b = (self.loss(a, b + h, x, y) - self.loss(a, b, x, y)) / h
        return gradient_a, gradient_b

    def weight_update(self, x, y, lr=0.01):
        grad_a, grad_b = self.gradient(x, y)
        self.a = self.a - lr * grad_a
        self.b = self.b - lr * grad_b

## 데이터 하나에 대하여 구해보기!
input data가 $(x,y) = (3,1)$로 주어질때 한번의 업데이트 이후 a와 b 값을 확인하세요!

In [2]:
model = Model(2, 1)
x, y = 3, 1
print(f"Before : a = {model.a:.2f}, b = {model.b:.2f}")
for epoch in range(1):
    model.weight_update(x, y)
print(f"After  : a = {model.a:.2f}, b = {model.b:.2f}")

Before : a = 2.00, b = 1.00
After  : a = 1.64, b = 0.88


## 10000번의 학습 이후 y_hat확인
input data가 $(x,y) = (3,1)$로 주어질때 10000번의 weight 업데이트를 진행하고 y와 y_hat이 같은지 확인해 보세요!

In [3]:
model = Model(2, 1)
x, y = 3, 1
print(f"Before : a = {model.a:.2f}, b = {model.b:.2f}")
for epoch in range(10000):
    model.weight_update(x, y)
print(f"After  : a = {model.a:.2f}, b = {model.b:.2f}")
print(f"y = {y:.2f}, y_hat = {model(x):.2f}")

Before : a = 2.00, b = 1.00
After  : a = 0.20, b = 0.40
y = 1.00, y_hat = 1.00


# Model2
Model Class를 상속받아 데이터가 2개 이상으로 들어오는 경우 Gradient의 평균을 계산하여 업데이트하는 모델을 만드세요!

In [4]:
class BatchModel(Model):
    def __init__(self, a, b):
        super().__init__(a, b)

    def batch_gradient(self, X, Y, h=1e-8):
        batch_gradient_a, batch_gradient_b = [], []
        for x, y in zip(X, Y):
            gradient_a, gradient_b = self.gradient(x, y)
            batch_gradient_a.append(gradient_a)
            batch_gradient_b.append(gradient_b)
        batch_gradient_a = sum(batch_gradient_a) / len(batch_gradient_a)
        batch_gradient_b = sum(batch_gradient_b) / len(batch_gradient_b)
        return batch_gradient_a, batch_gradient_b

    def weight_update(self, X, Y, lr=0.01):
        grad_a, grad_b = self.batch_gradient(X, Y)
        self.a = self.a - lr * grad_a
        self.b = self.b - lr * grad_b

## 데이터 하나에 대하여 구해보기!
input data가 $(𝑥,𝑦)=\{(3,1),(-1,5)\}$로 주어질때 한번의 업데이트 이후 a와 b 값을 확인하세요!

In [5]:
batch_model = BatchModel(2, 1)
X = [3, -1]
Y = [1, 5]
print(f"Before : a = {batch_model.a:.2f}, b = {batch_model.b:.2f}")
for epoch in range(1):
    batch_model.weight_update(X, Y)
print(f"After  : a = {batch_model.a:.2f}, b = {batch_model.b:.2f}")

Before : a = 2.00, b = 1.00
After  : a = 1.76, b = 1.00


## 10000번의 학습 이후 y_hat확인
input data가 (𝑥,𝑦)={(3,1),(−1,5)}로 주어질때 한번의 업데이트 이후 a와 b 값을 확인하세요! \
우리의 모델은 선형모델로 일차직선을 의미하는데 두 점이 주어져 유일하게 한 직선을 결정할 수 있어요 \
weight a와 b가 두 점을 지나는 유일한 직선을 표현할 수 있는지 확인해 보세요!

In [6]:
batch_model = BatchModel(2, 1)
X = [3, -1]
Y = [1, 5]
print(f"Before : a = {batch_model.a:.2f}, b = {batch_model.b:.2f}")
for epoch in range(10000):
    batch_model.weight_update(X, Y)
print(f"After  : a = {batch_model.a:.2f}, b = {batch_model.b:.2f}")
print(f"y = {y:.2f}, y_hat = {batch_model(x):.2f}")

Before : a = 2.00, b = 1.00
After  : a = -1.00, b = 4.00
y = 1.00, y_hat = 1.00


# (보충) 학습 시각화
BatchModel을 상속받아 weight의 update마다 weight의 변화를 시각화

In [7]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

class VisualModel(BatchModel):
    def __init__(self, a, b):
        super().__init__(a, b)

    def show_update(self, X, Y, epoch=500, lr=0.01, is_grid=True):
        def init():
            ax.set_xlim(min(X) - 3, max(X) + 3)
            ax.set_ylim(min(Y) - 3, max(Y) + 3)
            ln.set_data([], [])
            return (ln,)

        def update(epo):
            ax.set_title(f"epoch : {epo}")
            x = np.linspace(int(min(X)) - 5, int(max(X)) + 5, 128)
            y = self.a * x + self.b
            ln.set_data(x, y)
            self.weight_update(X, Y, lr)
            return (ln,)

        fig, ax = plt.subplots()
        (ln,) = plt.plot([], [], "r-")
        ani = FuncAnimation(fig, update, frames=range(0, epoch, 5), init_func=init)
        plt.plot(X, Y, "o")
        if is_grid:
            plt.grid(True)
            plt.xticks(range(int(min(X)) - 3, int(max(X)) + 3))
            plt.yticks(range(int(min(Y)) - 3, int(max(Y)) + 3))
        plt.close()
        return HTML(ani.to_html5_video())


## 문제 2-2 시각화
epoch, lr의 값을 변화시키면서 학습이 어떻게 이루어지는지 관찰해보세요!!

In [8]:
vmodel = VisualModel(2, 1)
X = [3, -1]
Y = [1, 5]
vmodel.show_update(X, Y, epoch=500, lr=0.01)

## 데이터의 수가 많을때 시각화

In [9]:
vmodel = VisualModel(2, 1)
X = np.linspace(-2, 7, 128)
Y = (5 * X + 2) + np.random.normal(0, 5, 128)
vmodel.show_update(X, Y, is_grid=False)

# Version
- python = '3.8'
- numpy = '1.21.5'
- matplotlib = '3.6.2'
- IPython = '8.9.0'